In [ ]:

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import PolynomialFeatures


In [ ]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')


In [ ]:
df1.shape

In [ ]:
label_encoder = LabelEncoder()
categorical_cols = df1.select_dtypes(include=['object']).columns.tolist()
categorical_colstest = df2.select_dtypes(include=['object']).columns.tolist()

# You can also encode labels to identify categorical columns
for col in categorical_cols:
    df1[col] = label_encoder.fit_transform(df1[col])

for col in categorical_colstest:
    df2[col] = label_encoder.fit_transform(df2[col])


In [ ]:
print(df1.shape)

In [ ]:
df1_onehot = pd.get_dummies(df1,)
df2_onehot = pd.get_dummies(df2)

In [ ]:
df1_onehot = pd.get_dummies(df1,drop_first='true')
df2_onehot = pd.get_dummies(df2,drop_first= 'true')

In [ ]:
common_columns = df1_onehot.columns.intersection(df2_onehot.columns)

In [ ]:
X = df1_onehot.loc[:, df1_onehot.columns != 'price_doc']
y = df1_onehot[['price_doc']]


In [ ]:
y = df1_onehot[['price_doc']]

In [ ]:
X_train = df1_onehot[common_columns]
X_test = df2_onehot[common_columns]

In [ ]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_train)
X_testscaled = scaler.fit_transform(X_test)

In [ ]:
scaler = StandardScaler()

# Fit and transform the training data
X_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_testscaled = scaler.transform(X_test)

In [ ]:
n_components = 40  # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(X_filtered)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(X_test_high_variance)

In [ ]:
X_pca.shape

In [ ]:
# original_feature_indices = np.arange(X.shape[1])
threshold_value = 0.001
variance_filter = VarianceThreshold(threshold=threshold_value)

# Fit and transform the feature matrix
X_filtered = variance_filter.fit_transform(X_scaled)

X_test_high_variance = variance_filter.transform(X_testscaled)


In [ ]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=25)
sfs.fit(X_scaled, y)
X_selected = sfs.transform(X_scaled)
# Print the names of the selected features
print("Selected Features:", sfs.get_feature_names_out())




In [ ]:
X_filtered.shape

In [ ]:
X_test_high_variance.shape

In [ ]:
X_pca.shape

In [ ]:
df2_onehot_pca.shape

In [ ]:
reg2 = LinearRegression().fit(X_filtered, y)
print(reg2.score(X_filtered, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(X_test_high_variance)

In [ ]:
poly = PolynomialFeatures(2, interaction_only=True)
X2 = poly.fit_transform(X_pca)
test2 = poly.fit_transform(df2_onehot_pca)
print(X2.shape)
print(poly.get_feature_names_out())
reg2 = LinearRegression().fit(X2, y)
print(reg2.score(X2, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(test2)

In [ ]:
df3.shape

In [ ]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions28.csv', index=False)